In [ ]:
# =========================================
# Predicción de temperatura futura con LSTM
# =========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib

# 1. Cargar datos correctamente
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8")

# Quitar espacios extra en nombres de columnas
df.columns = df.columns.str.strip()

print("✅ Columnas detectadas:", df.columns)

# 2. Procesar columna de tiempo si existe
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# 3. Convertir a numérico y limpiar valores inválidos
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Verificar cantidad de filas
print(f"📊 Total de filas luego de limpiar: {len(df)}")

# 4. Escalado de todas las variables de entrada
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])

# Guardar scaler para uso futuro
joblib.dump(scaler, "scaler_clima.pkl")

# 5. Crear secuencias de tiempo
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

# Usaremos 24 pasos (por ejemplo, 24 horas si los datos son horarios)
n_pasos = 24
# Buscar el índice de la columna 'ts' (temperatura) en el escalado
columna_objetivo = list(cols).index('ts')

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)

print(f"✅ Secuencias creadas: X={X.shape}, y={y.shape}")

# 6. Dividir en entrenamiento y test
porc_entrenamiento = 0.8
n_train = int(len(X) * porc_entrenamiento)

X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# 7. Construcción del modelo LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1))  # salida: temperatura futura

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# 8. Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluación del modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"📉 MAE en test: {mae:.2f} °C")

# 10. Predicción futura usando la última ventana
ultima_secuencia = X[-1].reshape(1, n_pasos, X.shape[2])
prediccion_escalada = model.predict(ultima_secuencia)[0][0]

# 11. Invertir el escalado para obtener la temperatura real
dummy = np.zeros((1, scaled_data.shape[1]))
dummy[0, columna_objetivo] = prediccion_escalada
prediccion_real = scaler.inverse_transform(dummy)[0, columna_objetivo]

print(f"🌡️ Temperatura predicha a futuro: {prediccion_real:.2f} °C")

# 12. Guardar modelo y scaler
model.save("modelo_lstm_clima.h5")
print("✅ Modelo guardado como 'modelo_lstm_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")

✅ Columnas detectadas: Index(['momento', 'ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas',
       'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h',
       'deltaPresion1h', 'humedadRelativaCambio'],
      dtype='object')
📊 Total de filas luego de limpiar: 96566
✅ Secuencias creadas: X=(96542, 24, 15), y=(96542,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - loss: 0.0238 - mae: 0.0956 - val_loss: 0.0025 - val_mae: 0.0431
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0065 - mae: 0.0594 - val_loss: 0.0012 - val_mae: 0.0272
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0062 - mae: 0.0575 - val_loss: 5.9358e-04 - val_mae: 0.0195
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0059 - mae: 0.0560 - val_loss: 0.0017 - val_mae: 0.0325
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0057 - mae: 0.0548 - val_loss: 9.9663e-04 - val_mae: 0.0248
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0055 - mae: 0.0540 - val_loss: 5.3553e-04 - val_mae: 0.0182
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0054 - mae: 0.0531 - val_loss: 0.0012 - val_mae: 0.0295
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0055 - mae: 0.0534 - val_loss: 8.5169e-04 - val_mae: 0.0223
Epoch 9/

🌡️ Temperatura predicha a futuro: 14.09 °C
✅ Modelo guardado como 'modelo_lstm_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'


In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 716.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
# =========================================
# Predicción de temperatura futura con GRU
# =========================================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model_gru = Sequential()
model_gru.add(GRU(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])))
model_gru.add(Dropout(0.2))
model_gru.add(GRU(64))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1))

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_gru = model_gru.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_gru, mae_gru = model_gru.evaluate(X_test, y_test)
print(f"📉 MAE GRU: {mae_gru:.2f} °C")

model_gru.save("modelo_gru_clima.h5")
print("✅ Modelo GRU guardado como 'modelo_gru_clima.h5'")


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0242 - mae: 0.1005 - val_loss: 0.0028 - val_mae: 0.0449
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0066 - mae: 0.0595 - val_loss: 0.0010 - val_mae: 0.0247
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0058 - mae: 0.0557 - val_loss: 9.8220e-04 - val_mae: 0.0251
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0056 - mae: 0.0548 - val_loss: 9.9329e-04 - val_mae: 0.0256
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0054 - mae: 0.0533 - val_loss: 9.4119e-04 - val_mae: 0.0241
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0052 - mae: 0.0525 - val_loss: 6.2185e-04 - val_mae: 0.0201
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0052 - mae: 0.0521 - val_loss: 9.3193e-04 - val_mae: 0.0241
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0053 - mae: 0.0527 - val_loss: 5.6385e-04 - val_mae: 0.0184


📉 MAE GRU: 0.01 °C
✅ Modelo GRU guardado como 'modelo_gru_clima.h5'


In [ ]:
# =========================================
# Modelo híbrido CNN + LSTM
# =========================================

from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model_cnn_lstm = Sequential()
model_cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_pasos, X.shape[2])))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(LSTM(64))
model_cnn_lstm.add(Dropout(0.3))
model_cnn_lstm.add(Dense(1))

model_cnn_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_cnnlstm = model_cnn_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_cnnlstm, mae_cnnlstm = model_cnn_lstm.evaluate(X_test, y_test)
print(f"📉 MAE CNN+LSTM: {mae_cnnlstm:.2f} °C")

model_cnn_lstm.save("modelo_cnn_lstm_clima.h5")
print("✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.0360 - mae: 0.1170 - val_loss: 0.0017 - val_mae: 0.0326
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0095 - mae: 0.0711 - val_loss: 0.0027 - val_mae: 0.0414
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0092 - mae: 0.0693 - val_loss: 0.0013 - val_mae: 0.0278
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0089 - mae: 0.0677 - val_loss: 0.0012 - val_mae: 0.0266
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0083 - mae: 0.0658 - val_loss: 0.0019 - val_mae: 0.0329
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0083 - mae: 0.0657 - val_loss: 0.0014 - val_mae: 0.0314
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0083 - mae: 0.0651 - val_loss: 8.9853e-04 - val_mae: 0.0229
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0080 - mae: 0.0642 - val_loss: 7.8389e-04 - val_mae: 0.0215
Epoch 9/100
2414

📉 MAE CNN+LSTM: 0.02 °C
✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'


In [ ]:
# =========================================
# Modelo basado en Transformer simple
# =========================================

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Flatten
import tensorflow as tf
inputs = tf.keras.Input(shape=(n_pasos, X.shape[2]))
x = MultiHeadAttention(num_heads=4, key_dim=16)(inputs, inputs)
x = LayerNormalization(epsilon=1e-6)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1)(x)

model_transformer = tf.keras.Model(inputs, outputs)
model_transformer.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

history_trans = model_transformer.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

loss_trans, mae_trans = model_transformer.evaluate(X_test, y_test)
print(f"📉 MAE Transformer: {mae_trans:.2f} °C")

model_transformer.save("modelo_transformer_clima.h5")
print("✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'")


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.1362 - mae: 0.2151 - val_loss: 0.0280 - val_mae: 0.1340
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0300 - mae: 0.1266 - val_loss: 0.0097 - val_mae: 0.0763
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0273 - mae: 0.1207 - val_loss: 0.0057 - val_mae: 0.0601
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.0262 - mae: 0.1183 - val_loss: 0.0035 - val_mae: 0.0486
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.0255 - mae: 0.1160 - val_loss: 0.0038 - val_mae: 0.0494
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0248 - mae: 0.1139 - val_loss: 0.0076 - val_mae: 0.0694
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0245 - mae: 0.1131 - val_loss: 0.0067 - val_mae: 0.0655
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.0239 - mae: 0.1108 - val_loss: 0.0049 - val_mae: 0.0591
Epoch 9/100
2414/2414 ━━━━━━━━━

📉 MAE Transformer: 0.02 °C
✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'


In [ ]:
#codigo actualizado del primero
# =========================================
# Predicción de temperatura futura con LSTM (usando todas las columnas)
# =========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib
import math

# 1. Cargar datos
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8", low_memory=False)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# 2. Procesar columna de tiempo
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    df = df.dropna(subset=['momento'])
    df = df.sort_values('momento')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# 3. Limpieza de datos
# Convertir todas las columnas numéricas correctamente
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Reemplazar valores absurdos (muy grandes o pequeños)
df = df.replace([np.inf, -np.inf], np.nan)
df[cols] = df[cols].apply(lambda x: np.where(np.abs(x) > 1e6, np.nan, x))  # filtra extremos
df = df.dropna()

print(f"✅ Datos limpios: {len(df)} filas")
print("✅ Columnas utilizadas:", list(cols))

# 4. Escalado
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])
joblib.dump(scaler, "scaler_clima.pkl")

# 5. Crear secuencias
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

n_pasos = 24
columna_objetivo = list(cols).index('ts')  # Temperatura superficial

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)
print(f"📈 Secuencias creadas: X={X.shape}, y={y.shape}")

# 6. División train/test
n_train = int(len(X) * 0.8)
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# 7. Modelo LSTM
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# 8. Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluación
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 MÉTRICAS DEL MODELO:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R²   = {r2:.4f}")

# 10. Predicción 1–6 horas futuras
ultima_secuencia = X[-1].copy()
predicciones_futuras = []

for h in range(6):
    prediccion_escalada = model.predict(ultima_secuencia.reshape(1, n_pasos, X.shape[2]))[0][0]
    predicciones_futuras.append(prediccion_escalada)

    # Actualizar ventana
    nueva_fila = ultima_secuencia[-1].copy()
    nueva_fila[columna_objetivo] = prediccion_escalada
    ultima_secuencia = np.vstack([ultima_secuencia[1:], nueva_fila])

# 11. Invertir el escalado
dummy = np.zeros((len(predicciones_futuras), scaled_data.shape[1]))
dummy[:, columna_objetivo] = predicciones_futuras
predicciones_reales = scaler.inverse_transform(dummy)[:, columna_objetivo]

print("\n🌡️ PREDICCIONES FUTURAS (próximas 6 horas):")
for i, temp in enumerate(predicciones_reales, start=1):
    print(f"➡️ +{i}h: {temp:.2f} °C")

# 12. Guardar modelo y scaler
model.save("modelo_lstm_clima.h5")
print("\n✅ Modelo guardado como 'modelo_lstm_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")


✅ Datos limpios: 82734 filas
✅ Columnas utilizadas: ['ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas', 'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h', 'deltaPresion1h', 'humedadRelativaCambio']
📈 Secuencias creadas: X=(82710, 24, 15), y=(82710,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0300 - mae: 0.1028 - val_loss: 0.0025 - val_mae: 0.0421
Epoch 2/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0073 - mae: 0.0632 - val_loss: 7.0438e-04 - val_mae: 0.0213
Epoch 3/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0064 - mae: 0.0586 - val_loss: 0.0037 - val_mae: 0.0492
Epoch 4/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0064 - mae: 0.0583 - val_loss: 7.5404e-04 - val_mae: 0.0228
Epoch 5/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0062 - mae: 0.0573 - val_loss: 6.9317e-04 - val_mae: 0.0202
Epoch 6/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0061 - mae: 0.0566 - val_loss: 8.8577e-04 - val_mae: 0.0231
Epoch 7/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0058 - mae: 0.0554 - val_loss: 7.6431e-04 - val_mae: 0.0212
Epoch 8/100
2068/2068 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0057 - mae: 0.0548 - val_loss: 0.0013 - val_mae: 0.0260
Epoc


🌡️ PREDICCIONES FUTURAS (próximas 6 horas):
➡️ +1h: 16.25 °C
➡️ +2h: 16.37 °C
➡️ +3h: 16.44 °C
➡️ +4h: 16.47 °C
➡️ +5h: 16.50 °C
➡️ +6h: 16.53 °C

✅ Modelo guardado como 'modelo_lstm_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'


In [ ]:
#codigo actualizado del segundo
# =========================================
# Predicción de temperatura futura con GRU
# =========================================

import pandas as pd
import numpy as np
import math
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ===============================
# 1️⃣ Cargar y limpiar datos
# ===============================
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8", low_memory=False)
df.columns = df.columns.str.strip()

# Procesar la columna de tiempo
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    df = df.dropna(subset=['momento'])
    df = df.sort_values('momento')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# Convertir todas las columnas numéricas
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan)

# Eliminar valores absurdamente grandes
df[cols] = df[cols].apply(lambda x: np.where(np.abs(x) > 1e6, np.nan, x))
df = df.dropna()

print(f"✅ Datos limpios: {len(df)} filas")
print("✅ Columnas utilizadas:", list(cols))

# ===============================
# 2️⃣ Escalado de todas las columnas
# ===============================
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])
joblib.dump(scaler, "scaler_clima.pkl")

# ===============================
# 3️⃣ Crear secuencias de tiempo
# ===============================
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

n_pasos = 24  # 24 horas previas
columna_objetivo = list(cols).index('ts')  # temperatura superficial

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)
print(f"📈 Secuencias creadas: X={X.shape}, y={y.shape}")

# ===============================
# 4️⃣ División de entrenamiento y test
# ===============================
n_train = int(len(X) * 0.8)
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# ===============================
# 5️⃣ Construcción del modelo GRU
# ===============================
model_gru = Sequential([
    GRU(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(1)
])

model_gru.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# ===============================
# 6️⃣ Entrenamiento
# ===============================
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history_gru = model_gru.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# ===============================
# 7️⃣ Evaluación del modelo
# ===============================
y_pred = model_gru.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 MÉTRICAS DEL MODELO GRU:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R²   = {r2:.4f}")

# ===============================
# 8️⃣ Predicción futura (1–6 horas adelante)
# ===============================
horas_a_predecir = int(input("\n⏩ ¿Cuántas horas deseas predecir (1–6)? "))
horas_a_predecir = max(1, min(horas_a_predecir, 6))  # limitar entre 1 y 6

ultima_secuencia = X[-1].copy()
predicciones_futuras = []

for h in range(horas_a_predecir):
    prediccion_escalada = model_gru.predict(ultima_secuencia.reshape(1, n_pasos, X.shape[2]))[0][0]
    predicciones_futuras.append(prediccion_escalada)

    # Actualizar la secuencia
    nueva_fila = ultima_secuencia[-1].copy()
    nueva_fila[columna_objetivo] = prediccion_escalada
    ultima_secuencia = np.vstack([ultima_secuencia[1:], nueva_fila])

# Invertir el escalado
dummy = np.zeros((len(predicciones_futuras), scaled_data.shape[1]))
dummy[:, columna_objetivo] = predicciones_futuras
predicciones_reales = scaler.inverse_transform(dummy)[:, columna_objetivo]

print("\n🌡️ PREDICCIONES FUTURAS (temperatura ts):")
for i, temp in enumerate(predicciones_reales, start=1):
    print(f"➡️ +{i}h: {temp:.2f} °C")

# ===============================
# 9️⃣ Guardar modelo y scaler
# ===============================
model_gru.save("modelo_gru_clima.h5")
print("\n✅ Modelo GRU guardado como 'modelo_gru_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")


✅ Datos limpios: 3010374 filas
✅ Columnas utilizadas: ['ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas', 'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h', 'deltaPresion1h', 'humedadRelativaCambio']
📈 Secuencias creadas: X=(3010350, 24, 15), y=(3010350,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 491s 7ms/step - loss: 0.0065 - mae: 0.0551 - val_loss: 5.9505e-04 - val_mae: 0.0194
Epoch 2/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 488s 6ms/step - loss: 0.0050 - mae: 0.0492 - val_loss: 8.2456e-04 - val_mae: 0.0226
Epoch 3/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 488s 6ms/step - loss: 0.0048 - mae: 0.0477 - val_loss: 0.0011 - val_mae: 0.0252
Epoch 4/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 490s 7ms/step - loss: 0.0071 - mae: 0.0614 - val_loss: 0.0060 - val_mae: 0.0593
Epoch 5/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 489s 7ms/step - loss: 0.0102 - mae: 0.0760 - val_loss: 0.0066 - val_mae: 0.0624
Epoch 6/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 490s 7ms/step - loss: 0.0107 - mae: 0.0780 - val_loss: 0.0076 - val_mae: 0.0690
Epoch 7/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 490s 7ms/step - loss: 0.0109 - mae: 0.0788 - val_loss: 0.0091 - val_mae: 0.0736
Epoch 8/100
75259/75259 ━━━━━━━━━━━━━━━━━━━━ 492s 7ms/step - loss: 0.0113 - mae: 0.0802 - val_loss: 0.0101 - val_mae:


🌡️ PREDICCIONES FUTURAS (temperatura ts):
➡️ +1h: 8.80 °C

✅ Modelo GRU guardado como 'modelo_gru_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'


In [ ]:
#mejora del tercer modelo
# =========================================
# Modelo híbrido CNN + LSTM mejorado
# =========================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import math

# ==============================
# 1️⃣ Cargar y preparar los datos
# ==============================
df = pd.read_csv("dataset_ml.csv", sep=";")

# Convertir columna de tiempo
df["momento"] = pd.to_datetime(df["momento"], errors="coerce")

# Eliminar filas con valores nulos o corruptos
df = df.dropna()

# Usamos TODAS las columnas numéricas (excepto 'momento')
features = df.select_dtypes(include=[np.number]).columns
data = df[features].values

# Escalamos todos los valores entre 0 y 1
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# =====================================
# 2️⃣ Crear secuencias para entrenamiento
# =====================================
n_pasos = 24  # 24 horas de historial para predecir
horizonte = int(input("⏱️ Ingresa cuántas horas hacia el futuro deseas predecir (1-6): "))
horizonte = max(1, min(6, horizonte))  # limitar entre 1 y 6

X, y = [], []
for i in range(n_pasos, len(data_scaled) - horizonte):
    X.append(data_scaled[i - n_pasos:i])
    # variable objetivo: temperatura futura (columna 'ts')
    y.append(data_scaled[i + horizonte, list(features).index("ts")])

X, y = np.array(X), np.array(y)

# ==============================
# 3️⃣ División de datos
# ==============================
porcentaje_entrenamiento = 0.8
n_entrenamiento = int(len(X) * porcentaje_entrenamiento)

X_train, X_test = X[:n_entrenamiento], X[n_entrenamiento:]
y_train, y_test = y[:n_entrenamiento], y[n_entrenamiento:]

# ==============================
# 4️⃣ Modelo CNN + LSTM
# ==============================
model_cnn_lstm = Sequential()
model_cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_pasos, X.shape[2])))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(LSTM(64))
model_cnn_lstm.add(Dropout(0.3))
model_cnn_lstm.add(Dense(1))

model_cnn_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# ==============================
# 5️⃣ Entrenamiento
# ==============================
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_cnnlstm = model_cnn_lstm.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# ==============================
# 6️⃣ Evaluación
# ==============================
y_pred = model_cnn_lstm.predict(X_test)

# Desescalar resultados (para obtener temperatura real)
idx_temp = list(features).index("ts")
temp_min, temp_max = scaler.data_min_[idx_temp], scaler.data_max_[idx_temp]
y_pred_real = y_pred * (temp_max - temp_min) + temp_min
y_test_real = y_test * (temp_max - temp_min) + temp_min

# Métricas
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = math.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n📊 Resultados del modelo CNN + LSTM:")
print(f"MAE  : {mae:.2f} °C")
print(f"RMSE : {rmse:.2f} °C")
print(f"R²   : {r2:.3f}")

# ==============================
# 7️⃣ Guardar modelo
# ==============================
model_cnn_lstm.save("modelo_cnn_lstm_clima.h5")
print("✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'")

# ==============================
# 8️⃣ Predicción futura (horas siguientes)
# ==============================
ultimos_datos = data_scaled[-n_pasos:]
entrada = np.expand_dims(ultimos_datos, axis=0)
prediccion_futura = []

for i in range(horizonte):
    pred = model_cnn_lstm.predict(entrada)
    prediccion_futura.append(pred[0, 0])
    # desplazamos ventana y agregamos nueva predicción
    nueva_fila = entrada[0, -1, :].copy()
    nueva_fila[idx_temp] = pred[0, 0]
    entrada = np.append(entrada[:, 1:, :], [[nueva_fila]], axis=1)

# Desescalar las predicciones futuras
prediccion_futura_real = np.array(prediccion_futura) * (temp_max - temp_min) + temp_min

print(f"\n🌤️ Predicciones de temperatura para las próximas {horizonte} horas:")
for i, temp in enumerate(prediccion_futura_real, start=1):
    print(f"  +{i}h → {temp:.2f} °C")


⏱️ Ingresa cuántas horas hacia el futuro deseas predecir (1-6): 1


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 89s 5ms/step - loss: 0.0012 - mae: 0.0214 - val_loss: 5.2026e-05 - val_mae: 0.0058
Epoch 2/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 3.1310e-04 - mae: 0.0125 - val_loss: 4.6301e-05 - val_mae: 0.0048
Epoch 3/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 2.9916e-04 - mae: 0.0121 - val_loss: 5.5929e-05 - val_mae: 0.0057
Epoch 4/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 2.9550e-04 - mae: 0.0120 - val_loss: 2.4413e-05 - val_mae: 0.0036
Epoch 5/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 2.9218e-04 - mae: 0.0119 - val_loss: 2.9099e-05 - val_mae: 0.0039
Epoch 6/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 82s 5ms/step - loss: 2.8799e-04 - mae: 0.0118 - val_loss: 6.4152e-05 - val_mae: 0.0063
Epoch 7/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 82s 5ms/step - loss: 2.8655e-04 - mae: 0.0118 - val_loss: 6.1514e-05 - val_mae: 0.0058
Epoch 8/100
15617/15617 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 2.8103e-04 - m


📊 Resultados del modelo CNN + LSTM:
MAE  : 0.13 °C
RMSE : 0.19 °C
R²   : 0.999
✅ Modelo CNN+LSTM guardado como 'modelo_cnn_lstm_clima.h5'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

🌤️ Predicciones de temperatura para las próximas 1 horas:
  +1h → 33.56 °C


In [ ]:
#mejora cuarto modelo
# =========================================
# Modelo basado en Transformer simple mejorado
# =========================================

import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Flatten
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# ==============================
# 1️⃣ Cargar y preparar los datos
# ==============================
df = pd.read_csv("dataset_ml.csv", sep=";")

# Convertir columna de tiempo
df["momento"] = pd.to_datetime(df["momento"], errors="coerce")

# Eliminar filas con valores nulos o corruptos
df = df.dropna()

# Usar TODAS las columnas numéricas excepto 'momento'
features = df.select_dtypes(include=[np.number]).columns
data = df[features].values

# Escalar todas las variables
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# =====================================
# 2️⃣ Crear secuencias para entrenamiento
# =====================================
n_pasos = 24  # 24 pasos (ej: 24 horas anteriores)
horizonte = int(input("⏱️ Ingresa cuántas horas hacia el futuro deseas predecir (1-6): "))
horizonte = max(1, min(6, horizonte))  # limitar entre 1 y 6

X, y = [], []
for i in range(n_pasos, len(data_scaled) - horizonte):
    X.append(data_scaled[i - n_pasos:i])
    # variable objetivo: temperatura superficial futura ("ts")
    y.append(data_scaled[i + horizonte, list(features).index("ts")])

X, y = np.array(X), np.array(y)

# ==============================
# 3️⃣ División de datos
# ==============================
porcentaje_entrenamiento = 0.8
n_entrenamiento = int(len(X) * porcentaje_entrenamiento)

X_train, X_test = X[:n_entrenamiento], X[n_entrenamiento:]
y_train, y_test = y[:n_entrenamiento], y[n_entrenamiento:]

# ==============================
# 4️⃣ Definir modelo Transformer
# ==============================
inputs = Input(shape=(n_pasos, X.shape[2]))
x = MultiHeadAttention(num_heads=4, key_dim=16)(inputs, inputs)
x = LayerNormalization(epsilon=1e-6)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1)(x)

model_transformer = Model(inputs, outputs)
model_transformer.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# ==============================
# 5️⃣ Entrenamiento
# ==============================
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_trans = model_transformer.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# ==============================
# 6️⃣ Evaluación del modelo
# ==============================
y_pred = model_transformer.predict(X_test)

# Desescalar temperatura (variable "ts")
idx_temp = list(features).index("ts")
temp_min, temp_max = scaler.data_min_[idx_temp], scaler.data_max_[idx_temp]
y_pred_real = y_pred * (temp_max - temp_min) + temp_min
y_test_real = y_test * (temp_max - temp_min) + temp_min

# Calcular métricas
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = math.sqrt(mean_squared_error(y_test_real, y_pred_real))
r2 = r2_score(y_test_real, y_pred_real)

print("\n📊 Resultados del modelo Transformer:")
print(f"MAE  : {mae:.2f} °C")
print(f"RMSE : {rmse:.2f} °C")
print(f"R²   : {r2:.3f}")

# ==============================
# 7️⃣ Guardar modelo
# ==============================
model_transformer.save("modelo_transformer_clima.h5")
print("✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'")

# ==============================
# 8️⃣ Predicción futura (1–6 horas)
# ==============================
ultimos_datos = data_scaled[-n_pasos:]
entrada = np.expand_dims(ultimos_datos, axis=0)
prediccion_futura = []

for i in range(horizonte):
    pred = model_transformer.predict(entrada)
    prediccion_futura.append(pred[0, 0])
    # Actualizar la ventana
    nueva_fila = entrada[0, -1, :].copy()
    nueva_fila[idx_temp] = pred[0, 0]
    entrada = np.append(entrada[:, 1:, :], [[nueva_fila]], axis=1)

# Desescalar predicciones futuras
prediccion_futura_real = np.array(prediccion_futura) * (temp_max - temp_min) + temp_min

print(f"\n🌤️ Predicciones de temperatura para las próximas {horizonte} horas:")
for i, temp in enumerate(prediccion_futura_real, start=1):
    print(f"  +{i}h → {temp:.2f} °C")


⏱️ Ingresa cuántas horas hacia el futuro deseas predecir (1-6): 1
Epoch 1/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 229s 3ms/step - loss: 0.0048 - mae: 0.0351 - val_loss: 0.0056 - val_mae: 0.0511
Epoch 2/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 220s 2ms/step - loss: 8.0168e-04 - mae: 0.0199 - val_loss: 0.0056 - val_mae: 0.0505
Epoch 3/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 217s 2ms/step - loss: 7.7594e-04 - mae: 0.0195 - val_loss: 0.0051 - val_mae: 0.0482
Epoch 4/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 218s 2ms/step - loss: 7.6891e-04 - mae: 0.0194 - val_loss: 0.0053 - val_mae: 0.0482
Epoch 5/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 217s 2ms/step - loss: 7.6646e-04 - mae: 0.0193 - val_loss: 0.0042 - val_mae: 0.0456
Epoch 6/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 220s 2ms/step - loss: 7.6648e-04 - mae: 0.0193 - val_loss: 0.0039 - val_mae: 0.0413
Epoch 7/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 217s 2ms/step - loss: 7.6198e-04 - mae: 0.0192 - val_loss: 0.0048 - val_mae: 0.0454
Epoch 8/100
88738/88738 ━━━━━━━━━━━


📊 Resultados del modelo Transformer:
MAE  : 1.53 °C
RMSE : 2.22 °C
R²   : 0.919
✅ Modelo Transformer guardado como 'modelo_transformer_clima.h5'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step

🌤️ Predicciones de temperatura para las próximas 1 horas:
  +1h → 28.04 °C


In [ ]:
# =========================================
# Predicción de temperatura futura con LSTM (usando todas las columnas)
# =========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib
import math

# 1. Cargar datos
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8", low_memory=False)

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# 2. Procesar columna de tiempo
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    df = df.dropna(subset=['momento'])
    df = df.sort_values('momento')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# 3. Limpieza de datos
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan)
df[cols] = df[cols].apply(lambda x: np.where(np.abs(x) > 1e6, np.nan, x))
df = df.dropna()

print(f"✅ Datos limpios: {len(df)} filas")
print("✅ Columnas utilizadas:", list(cols))

# 4. Escalado
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])
joblib.dump(scaler, "scaler_clima.pkl")

# 5. Crear secuencias
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

n_pasos = 24
columna_objetivo = list(cols).index('ts')

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)
print(f"📈 Secuencias creadas: X={X.shape}, y={y.shape}")

# 6. División train/test
n_train = int(len(X) * 0.8)
X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# 7. Modelo LSTM
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# 8. Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluación
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n📊 MÉTRICAS DEL MODELO:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R²   = {r2:.4f}")

# 10. Predicción futura personalizada (1–6 horas)
try:
    horas_futuras = int(input("\n⏱️ Ingresa cuántas horas hacia el futuro deseas predecir (1–6): "))
except:
    horas_futuras = 6  # valor por defecto si el usuario no escribe bien
horas_futuras = max(1, min(6, horas_futuras))  # limitar entre 1 y 6

ultima_secuencia = X[-1].copy()
predicciones_futuras = []

for h in range(horas_futuras):
    prediccion_escalada = model.predict(ultima_secuencia.reshape(1, n_pasos, X.shape[2]))[0][0]
    predicciones_futuras.append(prediccion_escalada)

    # Actualizar ventana (deslizar una hora)
    nueva_fila = ultima_secuencia[-1].copy()
    nueva_fila[columna_objetivo] = prediccion_escalada
    ultima_secuencia = np.vstack([ultima_secuencia[1:], nueva_fila])

# 11. Invertir el escalado
dummy = np.zeros((len(predicciones_futuras), scaled_data.shape[1]))
dummy[:, columna_objetivo] = predicciones_futuras
predicciones_reales = scaler.inverse_transform(dummy)[:, columna_objetivo]

print(f"\n🌡️ PREDICCIONES FUTURAS (próximas {horas_futuras} horas):")
for i, temp in enumerate(predicciones_reales, start=1):
    print(f"➡️ +{i}h: {temp:.2f} °C")

# 12. Guardar modelo y scaler
model.save("modelo_lstm_clima.h5")
print("\n✅ Modelo guardado como 'modelo_lstm_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")


✅ Datos limpios: 3549541 filas
✅ Columnas utilizadas: ['ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas', 'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h', 'deltaPresion1h', 'humedadRelativaCambio']
📈 Secuencias creadas: X=(3549517, 24, 15), y=(3549517,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 657s 7ms/step - loss: 0.0059 - mae: 0.0524 - val_loss: 0.9370 - val_mae: 0.6098
Epoch 2/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 656s 7ms/step - loss: 0.0044 - mae: 0.0452 - val_loss: 0.8574 - val_mae: 0.5664
Epoch 3/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 656s 7ms/step - loss: 0.0042 - mae: 0.0444 - val_loss: 0.7791 - val_mae: 0.6011
Epoch 4/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 655s 7ms/step - loss: 0.0041 - mae: 0.0439 - val_loss: 0.4263 - val_mae: 0.3776
Epoch 5/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 652s 7ms/step - loss: 0.0041 - mae: 0.0436 - val_loss: 0.9551 - val_mae: 0.6201
Epoch 6/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 648s 7ms/step - loss: 0.0041 - mae: 0.0434 - val_loss: 2.6699 - val_mae: 1.0835
Epoch 7/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 651s 7ms/step - loss: 0.0041 - mae: 0.0433 - val_loss: 1.5962 - val_mae: 0.8012
Epoch 8/100
88738/88738 ━━━━━━━━━━━━━━━━━━━━ 649s 7ms/step - loss: 0.0041 - mae: 0.0433 - val_loss: 0.6616 - val_mae: 0.4971



🌡️ PREDICCIONES FUTURAS (próximas 1 horas):
➡️ +1h: 30.93 °C

✅ Modelo guardado como 'modelo_lstm_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'
